In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import random
import datetime
from haversine import haversine, Unit

In [2]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size

from torch import Tensor
from torch.nn import Linear
from torch_geometric.nn.conv import MessagePassing

#The weighted Graph Sage network, copied from PyG GraphSage code and GCN code
class WeightedSAGEConv(MessagePassing):
    def __init__(self, in_channels: int,
                 out_channels: int, bias: bool = True, **kwargs):  # yapf: disable
        kwargs.setdefault('aggr', 'add')
        super(WeightedSAGEConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels

        in_channels = (in_channels, in_channels)

        self.lin_l = Linear(in_channels[0], out_channels, bias=bias)
        self.lin_r = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x: Tensor, edge_index: Adj,
                normalize: Tensor) -> Tensor:
        
        x: OptPairTensor = (x, x)
   
        # propagate_type: (x: OptPairTensor)
        out = self.propagate(edge_index, x=x, norm = normalize)
        out = self.lin_l(out)

        x_r = x[1]
        out += self.lin_r(x_r)

        return out

    def message(self, x_j: Tensor, norm) -> Tensor:
        return x_j*norm.view(-1,1)

In [16]:
import datetime
#Put the file location
df = pd.read_csv('D:\\STUDIES\\BTP\\20_10_all2.csv')

#type casting
df.pm1_0 = df.pm1_0.astype(float)
df.pm2_5 = df.pm2_5.astype(float)
df.pm10 = df.pm10.astype(float)
df.lat = round(round(5*df.lat.astype(float),2)/5.0,3)
df.long= round(round(5*df.long.astype(float),2)/5.0,3)
df.pressure = df.pressure.astype(float)
df.temperature = df.temperature.astype(float)
df.humidity = df.humidity.astype(float)
df.dateTime = pd.to_datetime(df.dateTime)

print(len(df))
#Ensuring Delhi region and removing outliers from data
# df = df[(df.lat.astype(int) == 28) &(df.long.astype(int) == 77)]
# df = df[(df.pm1_0<=1500) & (df.pm2_5<=1500) & (df.pm10<=1500) & (df.pm1_0>=20) & (df.pm2_5>=30) & (df.pm10>=30)]
# df = df[(df.humidity<=60)&(df.humidity>=7)]
print(len(df))

In [17]:
#data points distribution
num = [ len(df[(df.dateTime.dt.hour == i)]) for i in range(24) ]
print(np.where(np.array(num) == np.max(np.array(num)))[0][0])
print(num)

length of Dataset: 88769


,Unnamed: 0,dateTime,deviceId,uid,pressure,temperature,humidity,lat,long,pm1_0,pm2_5,pm10
0,0,2020-10-20 00:00:00+05:30,0000000024568afd,b0de5d1a-504c-4c26-86f8-f7202ab648bd,982.193437,33.18,23.709961,28.58,77.234,67.0,109.0,116.0
1,1,2020-10-20 00:00:00+05:30,0000000038861c77,41965d23-6f04-46de-a00e-d15998ad6949,978.037578,34.35,25.658203,28.58,77.228,56.0,91.0,99.0
2,2,2020-10-20 00:00:02+05:30,00000000078e6811,0e8cc005-4608-4fa5-b1a2-763b0880a87d,982.658125,34.16,24.464844,28.58,77.226,57.0,86.0,94.0
3,3,2020-10-20 00:00:02+05:30,00000000c37f0aa8,ecabd170-4090-43a0-a565-c4ca2bb9d7ac,982.586406,33.16,29.271484,28.58,77.230,61.0,93.0,103.0
4,4,2020-10-20 00:00:03+05:30,0000000024568afd,c4f1771d-6956-483b-ad2f-5cc894c972a9,982.081094,33.16,23.674805,28.58,77.234,66.0,110.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88764,88764,2020-10-20 23:59:54+05:30,00000000812d59d2,b1f02316-4723-42be-9a5c-cec278f3cf7b,982.412969,30.22,41.182617,28.58,77.228,77.0,131.0,144.0
88765,88765,2020-10-20 23:59:54+05:30,0000000038861c77,b27878be-b202-4f15-ad62-74605c02b563,978.070469,33.00,32.031250,28.58,77.234,80.0,131.0,141.0
88766,88766,2020-10-20 23:59:56+05:30,0000000038861c77,8f5a44b8-a3f3-4880-a5ec-2e7e8d659222,978.296875,33.12,32.016602,28.58,77.234,80.0,131.0,137.0
88767,88767,2020-10-20 23:59:56+05:30,00000000078e6811,c133e2b2-6af0-45eb-920b-df9645d8c574,984.762500,35.75,28.152344,28.58,77.228,77.0,111.0,125.0


In [30]:
df.iloc[81155].dateTime.hour

In [29]:
#--------------------------------Rounding @15min----------------------------------
df.dateTime = df.dateTime.dt.round('15min')
#---------------------------------Only PM10---------------------------------------
dfHourRaw = df[(df['dateTime'].dt.day == 20)][['dateTime','lat','long','pm1_0']]
dfHour = dfHourRaw #[['lat','long','pm1_0']]
meaned = dfHour.groupby(['dateTime','lat','long']).mean().reset_index()
meaned.pm1_0 = round(meaned.pm1_0,2)
#print(meaned)
#meaned[(meaned.pm1_0>45)|(meaned.pm1_0<25)]

In [22]:
#Converting Time to minutes
meaned.dateTime = meaned.dateTime.dt.hour*60+meaned.dateTime.dt.minute
meaned.dateTime %= 1440
meaned = meaned[(meaned.dateTime>=300) & (meaned.dateTime<=1350)]
meaned = meaned.sort_values(by = ['dateTime','lat','long'])
data = meaned.to_numpy()

#random sample of test data 
#------------------------------test_size = data_size * 1/20--------------------------------
testSet = random.sample(range(data.shape[0]),int(data.shape[0]/20))
testData = data[testSet]
data = np.delete(data,testSet,0)
print(testSet)
print(data)

[5321, 3195, 4739, 933, 4367, 5534, 4770, 5454, 1946, 4082, 1909, 927, 3614, 4502, 3034, 4318, 1624, 9, 5243, 4047, 3458, 146, 1406, 5389, 2674, 2310, 495, 1060, 4821, 3636, 390, 2916, 4132, 679, 985, 3371, 4254, 2963, 2095, 4021, 2423, 249, 1295, 5127, 2502, 4409, 2298, 4336, 3190, 1138, 137, 40, 1619, 4013, 3853, 4376, 2291, 4001, 5517, 3567, 3323, 461, 1492, 5593, 2901, 3237, 5013, 3285, 5291, 1284, 1233, 5617, 2990, 4742, 737, 2460, 2623, 2072, 3140, 1118, 1002, 5202, 3983, 1116, 2652, 3925, 3164, 3338, 5705, 92, 2648, 738, 4038, 4141, 2925, 4824, 1411, 3271, 1301, 12, 4089, 1247, 804, 1565, 3283, 2787, 2947, 1533, 3898, 5369, 2570, 4580, 116, 2997, 1489, 4583, 2271, 2662, 5740, 3873, 2440, 4981, 52, 3607, 4892, 1218, 2427, 537, 1747, 5591, 2989, 899, 2978, 229, 4426, 3621, 814, 3319, 4915, 1420, 2200, 3328, 5484, 3229, 3701, 4333, 3001, 2185, 5085, 1769, 488, 2815, 870, 2323, 3187, 4472, 2627, 1483, 424, 430, 2913, 4496, 4453, 648, 3207, 1935, 5026, 4928, 5072, 124, 2506, 454, 226

In [15]:
meaned.groupby(['dateTime']).size()[:50]

dateTime
300      59
315      28
330      81
345      94
360      75
375     102
390      81
405     132
420     145
435      93
450      70
465      82
480      79
495      93
510      85
525      62
540      44
555      20
570      26
585      41
600      54
615      80
630      57
645      91
660     107
675     104
690     102
705     113
720      90
735      92
750      95
765      63
780      55
795      78
810     103
825      83
840      76
855      82
870      65
885      96
900     115
915     129
930      74
945     106
960     125
975      96
990     116
1005    122
1020     57
1035     37
dtype: int64

In [9]:
%%time
from haversine import haversine, Unit

#----------------------------------------------2-3Km in space, 1-2.5hrs in time----------------------------------------
spaceD = data[:,1:3]
timeD = data[:,0]
dist = 1.0 #in Km, Time
edges = []
for i in range(len(data)):
    lim = 50
    count = 0
    se = []
    while(count<4):
        for j in range(len(data)):
            if(j in se):
                continue
            di = haversine(spaceD[i],spaceD[j])
            ti = int(abs(timeD[i]-timeD[j])/14.9)
            w = (1+di)*(1+di)*(1+di)*(1+ti/2)
            if ((ti<=8) & (w<=lim)):
                edges.append([j,i,1.0/w])
                count+=1
                se.append(j)
        lim+=50
edges = np.array(edges).T

Wall time: 43.8 s


In [10]:
data.shape, edges.shape

((2350, 4), (3, 109492))

In [11]:
# Find Normalization coefficients for edges in weighted graph sage
#-----------------------------------------------------------------------------------------------------------------------
def findNorm(tempEdges, x_size):
    weightSum = np.ones(x_size)
    ei = pd.DataFrame(tempEdges, columns = ['from','to','val'])
    ei = ei.groupby(['to']).agg('sum').reset_index()
    ei = ei[['from','to','val']]
    weightSum[ei.to.to_numpy().astype(int)] = ei.val.to_numpy()
    weightSum = torch.from_numpy(weightSum)
    normalize = torch.zeros(tempEdges.shape[0])
    for i,edge in enumerate(tempEdges):
        if(weightSum[int(edge[1])] == 0):
            print("Error in WeightedSageConv, as one of the nodes has no incoming edges to it.")
        normalize[i] = edge[2]*1.0/weightSum[int(edge[1])]    
    return normalize

#Mean Finder
#------------------------------------------------------------------------------------------------------------------------
def Meaner(dt):
    ei = dt.edge_index.to('cpu')
    if(type(ei) ==torch.Tensor):
        ei = ei.numpy()
    w = dt.norm.to('cpu')
    if(type(w) == torch.Tensor):
        w = w.numpy()
    n = dt.x.to('cpu')
    if(type(n) == torch.Tensor):
        n = n.numpy()
    print(ei.shape, w.shape, n.shape)
    if(n.ndim == 2):
        n = n[:,0]
    predVal = torch.zeros(n.shape[0], dtype = float)
    for i in range(ei.shape[1]):
        predVal[ei[1][i]] += n[ei[0][i]]*w[i]
    return predVal

data.shape, edges.shape

((2350, 4), (3, 109492))

In [12]:
X = data[:,3]
X2 = torch.from_numpy(X)
graph = Data(x = X2, edge_index = torch.from_numpy(edges[:2,:]).type(torch.LongTensor), norm = findNorm(edges.T,len(X)))
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
ei = graph.edge_index*1
wi = graph.norm
ei,wi = edges[:, ei[1,:] == 0],wi[ei[1,:] == 0]
ei.T,wi

(array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.00000000e+00, 0.00000000e+00, 3.03197873e-01],
        [2.00000000e+00, 0.00000000e+00, 2.65455556e-01],
        [3.00000000e+00, 0.00000000e+00, 9.20783084e-02],
        [4.00000000e+00, 0.00000000e+00, 1.06156993e-01],
        [5.00000000e+00, 0.00000000e+00, 3.96335839e-02],
        [6.00000000e+00, 0.00000000e+00, 4.87197201e-02],
        [7.00000000e+00, 0.00000000e+00, 2.48018719e-02],
        [3.00000000e+01, 0.00000000e+00, 3.24791183e-02],
        [3.10000000e+01, 0.00000000e+00, 6.13839895e-02],
        [3.20000000e+01, 0.00000000e+00, 1.26522732e-01],
        [3.30000000e+01, 0.00000000e+00, 2.02131915e-01],
        [3.90000000e+01, 0.00000000e+00, 9.48920492e-02],
        [4.00000000e+01, 0.00000000e+00, 5.00000000e-01],
        [4.10000000e+01, 0.00000000e+00, 1.51598936e-01],
        [4.20000000e+01, 0.00000000e+00, 1.32727778e-01],
        [4.30000000e+01, 0.00000000e+00, 4.60391542e-02],
        [4.400

In [14]:
mg = Meaner(graph)

(2, 109492) (109492,) (2350,)


In [15]:
ls = abs(mg-X2)>5
mg[ls], X2[ls]
F.mse_loss(mg,X2)**0.5

tensor(11.2002, dtype=torch.float64)

In [16]:
# Create Dataset/Subgraphs for training.
#------------------------------------------ validation samples = train Data * 1/10 ---------------------------------------------
def createDataset(d: np.array, edges: np.array, numGraphs:int = 50, device = 'cpu'):
    #d is PM10 values of train set, edges are edge list on this train set
    dataset = []
    trainSamples = int(len(d)/10)
    count = 0
    while(count<numGraphs):
        trainSet = random.sample(range(len(d)), trainSamples)
        trainSet = np.sort(trainSet)
        trainSet = torch.from_numpy(trainSet)
        #proper subset making and reshaping
        #removing edges coming out of validation nodes
        tempEdges = edges[:,[i not in trainSet for i in edges[0]]]
        #tempEdges = tempEdges[:, tempEdges[1].argsort()]
        subWeights = torch.reshape(torch.tensor(tempEdges[2,:], dtype = torch.float),(tempEdges.shape[1],1))
        subEdges = torch.tensor(tempEdges[:2,:], dtype =torch.long)
        
        #2 features of nodes, 1) PM10 value, 2)Presence
        subNodes = np.ones((d.shape[0],2))
        subNodes[:,0] = d.copy()
        subNodes[trainSet,:] = 0.0
        subNodes = torch.tensor(subNodes, dtype = torch.float)
        
        #print(d.shape[0])
        #normalization calculation
        norm = findNorm(tempEdges.T, d.shape[0])
        sample = Data(x = subNodes, edge_index = subEdges, train_set = trainSet, edge_attr = subWeights, norm = norm)
        sample.to(device)
        if((not sample.contains_isolated_nodes())):
            dataset.append(sample)
            print(count)
            count+=1
    return dataset

In [17]:
%%time
#------------------------------------Assuming Data is nx4 where last column is pollutant value-------------------------------
dataset = createDataset(X,edges,device = dev)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Wall time: 6min 11s


In [18]:
device = dev

In [19]:
X2 = torch.from_numpy(X)
X1 = X2.to(device)
X1.shape

torch.Size([2350])

r"""The GraphSAGE operator from the `"Inductive Representation Learning on
Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper
$$\mathbf{x}^{\prime}_i = \mathbf{W}_1 \mathbf{x}_i + \mathbf{W_2} \cdot
\mathrm{mean}_{j \in \mathcal{N(i)}} \mathbf{x}_j$$
Args:
    in_channels (int or tuple): Size of each input sample. A tuple
        corresponds to the sizes of source and target dimensionalities.
    out_channels (int): Size of each output sample.
    normalize (bool, optional): If set to :obj:`True`, output features
        will be :math:$\ell_2$-normalized, *i.e.*,
        :math:
        $$\frac{\mathbf{x}^{\prime}_i}
        {\| \mathbf{x}^{\prime}_i \|_2}$$.
        (default: :obj:`False`)
    bias (bool, optional): If set to :obj:`False`, the layer will not learn
        an additive bias. (default: :obj:`True`)
    **kwargs (optional): Additional arguments of
        :class:`torch_geometric.nn.conv.MessagePassing`.
"""

In [20]:
from torch_geometric.nn import SAGEConv
#Net of 2 x 28 x 14 x 5 x 1
#input 2-> mean GraphSage 10-> mean GraphSage 6-> Linear Layer 3-> Output 1
#       ->  max GraphSage  4   max  GraphSage 2
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1Mean = WeightedSAGEConv(2,24)
        self.conv1Max = SAGEConv(2,8)
        self.conv1Max.aggr = 'max'
        self.conv2Mean = WeightedSAGEConv(32,12)
        self.conv2Max = SAGEConv(32,4)
        self.conv2Max.aggr = 'max'
        self.conv3 = nn.Linear(16, 8)
        self.conv4 = nn.Linear(8, 5)
        self.conv5 = nn.Linear(5, 1)

    def forward(self,data):
        x, edge_index, norm = data.x.float(), data.edge_index, data.norm
        y = F.relu(self.conv1Mean(x, edge_index, norm))
        z = F.relu(self.conv1Max(x, edge_index))
        x = torch.cat((y,z),1)
        
        y = F.relu(self.conv2Mean(x, edge_index, norm))
        z = F.relu(self.conv2Max(x, edge_index))
        x = torch.cat((y,z),1)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.conv5(x)
        return torch.squeeze(x)

In [21]:
net = Net()
net.to(device)
net = net.float()
net.train()

Net(
  (conv1Mean): WeightedSAGEConv(
    (lin_l): Linear(in_features=2, out_features=24, bias=True)
    (lin_r): Linear(in_features=2, out_features=24, bias=False)
  )
  (conv1Max): SAGEConv(2, 8)
  (conv2Mean): WeightedSAGEConv(
    (lin_l): Linear(in_features=32, out_features=12, bias=True)
    (lin_r): Linear(in_features=32, out_features=12, bias=False)
  )
  (conv2Max): SAGEConv(32, 4)
  (conv3): Linear(in_features=16, out_features=8, bias=True)
  (conv4): Linear(in_features=8, out_features=5, bias=True)
  (conv5): Linear(in_features=5, out_features=1, bias=True)
)

In [22]:
def train(opt, dataset):
    net.train()
    opt.zero_grad()
    fv = dataset[0]
    output = net(fv)
    output = torch.reshape(output,(-1,))
    loss = F.mse_loss(output.float(),X1.float())
    for i in dataset[1:]:
        output = net(i)
        output = torch.reshape(output,(-1,))
        indLoss = F.mse_loss(output.float(),X1.float())
        loss = loss + indLoss
    loss.backward()
    opt.step()
    return loss.item()

In [23]:
import torch.optim as optim
optimizer1 = torch.optim.Adam(net.parameters(), lr = 0.1)
optimizer01 = torch.optim.Adam(net.parameters(), lr = 0.01)
optimizer001 = torch.optim.Adam(net.parameters(), lr = 0.001)

In [24]:
dataset[0].x.shape

torch.Size([2350, 2])

In [25]:
%%time
oz = optimizer001
for epoch in range(5000):
    loss = train(oz, dataset)
    print(epoch,": ",loss)

0 :  347644.53125
1 :  345111.03125
2 :  342604.5
3 :  340160.5625
4 :  337813.53125
5 :  336078.875
6 :  335770.78125
7 :  335445.125
8 :  335094.84375
9 :  334775.40625
10 :  334465.78125
11 :  334135.65625
12 :  333790.375
13 :  333454.53125
14 :  333116.9375
15 :  332767.1875
16 :  332402.46875
17 :  332019.59375
18 :  331616.34375
19 :  331208.40625
20 :  330788.875
21 :  330348.0625
22 :  329879.625
23 :  329380.4375
24 :  328849.5625
25 :  328286.96875
26 :  327691.0
27 :  327060.375
28 :  326393.8125
29 :  325689.34375
30 :  324945.125
31 :  324159.25
32 :  323329.5625
33 :  322453.6875
34 :  321529.21875
35 :  320553.6875
36 :  319524.28125
37 :  318438.09375
38 :  317292.25
39 :  316083.6875
40 :  314809.03125
41 :  313465.0625
42 :  312048.1875
43 :  310554.78125
44 :  308981.03125
45 :  307323.1875
46 :  305558.9375
47 :  303626.28125
48 :  301528.96875
49 :  299278.375
50 :  296904.28125
51 :  294402.375
52 :  291760.34375
53 :  288969.71875
54 :  286024.03125
55 :  282923

699 :  1859.7889404296875
700 :  1858.1910400390625
701 :  1856.5938720703125
702 :  1854.995361328125
703 :  1853.394287109375
704 :  1851.7919921875
705 :  1850.189208984375
706 :  1848.5841064453125
707 :  1846.9755859375
708 :  1845.3670654296875
709 :  1843.7578125
710 :  1842.1497802734375
711 :  1840.543212890625
712 :  1838.9364013671875
713 :  1837.3289794921875
714 :  1835.721923828125
715 :  1834.1136474609375
716 :  1832.5050048828125
717 :  1830.8958740234375
718 :  1829.2857666015625
719 :  1827.675048828125
720 :  1826.0645751953125
721 :  1824.453369140625
722 :  1822.84130859375
723 :  1821.22998046875
724 :  1819.6171875
725 :  1818.0045166015625
726 :  1816.3919677734375
727 :  1814.7783203125
728 :  1813.1634521484375
729 :  1811.5477294921875
730 :  1809.9307861328125
731 :  1808.313720703125
732 :  1806.6964111328125
733 :  1805.0814208984375
734 :  1803.4691162109375
735 :  1801.85546875
736 :  1800.240234375
737 :  1798.62744140625
738 :  1797.0120849609375
739 

1344 :  921.6165161132812
1345 :  920.7606811523438
1346 :  919.9073486328125
1347 :  919.0567626953125
1348 :  918.208740234375
1349 :  917.3636474609375
1350 :  916.5209350585938
1351 :  915.6810913085938
1352 :  914.843994140625
1353 :  914.0094604492188
1354 :  913.1777954101562
1355 :  912.3487548828125
1356 :  911.5222778320312
1357 :  910.69873046875
1358 :  909.877685546875
1359 :  909.0592041015625
1360 :  908.2435302734375
1361 :  907.4304809570312
1362 :  906.6200561523438
1363 :  905.8120727539062
1364 :  905.00732421875
1365 :  904.2046508789062
1366 :  903.4052734375
1367 :  902.6083374023438
1368 :  901.81396484375
1369 :  901.0224609375
1370 :  900.2332763671875
1371 :  899.4472045898438
1372 :  898.6636962890625
1373 :  897.882568359375
1374 :  897.1044921875
1375 :  896.3292846679688
1376 :  895.5567016601562
1377 :  894.7869262695312
1378 :  894.0197143554688
1379 :  893.25537109375
1380 :  892.4937744140625
1381 :  891.7344970703125
1382 :  890.9773559570312
1383 : 

1985 :  736.5133666992188
1986 :  736.4314575195312
1987 :  736.3468017578125
1988 :  736.2596435546875
1989 :  736.1721801757812
1990 :  736.0839233398438
1991 :  735.995361328125
1992 :  735.9065551757812
1993 :  735.817626953125
1994 :  735.7288818359375
1995 :  735.6395263671875
1996 :  735.5502319335938
1997 :  735.4614868164062
1998 :  735.3731079101562
1999 :  735.2861938476562
2000 :  735.1988525390625
2001 :  735.1112670898438
2002 :  735.0233764648438
2003 :  734.9356689453125
2004 :  734.8486938476562
2005 :  734.7620239257812
2006 :  734.67529296875
2007 :  734.5883178710938
2008 :  734.5001831054688
2009 :  734.4114990234375
2010 :  734.3233642578125
2011 :  734.235107421875
2012 :  734.1463623046875
2013 :  734.05712890625
2014 :  733.9668579101562
2015 :  733.8737182617188
2016 :  733.7787475585938
2017 :  733.6805419921875
2018 :  733.5774536132812
2019 :  733.4698486328125
2020 :  733.358642578125
2021 :  733.2455444335938
2022 :  733.1326904296875
2023 :  733.02416992

2627 :  706.037109375
2628 :  706.0032958984375
2629 :  705.9692993164062
2630 :  705.9354248046875
2631 :  705.901611328125
2632 :  705.8677978515625
2633 :  705.833984375
2634 :  705.8004760742188
2635 :  705.766357421875
2636 :  705.7327270507812
2637 :  705.6989135742188
2638 :  705.6646728515625
2639 :  705.6307983398438
2640 :  705.5968627929688
2641 :  705.5628662109375
2642 :  705.529296875
2643 :  705.495361328125
2644 :  705.4617919921875
2645 :  705.4280395507812
2646 :  705.39404296875
2647 :  705.360107421875
2648 :  705.3259887695312
2649 :  705.2911376953125
2650 :  705.2551879882812
2651 :  705.217529296875
2652 :  705.1785888671875
2653 :  705.1395263671875
2654 :  705.1011962890625
2655 :  705.064453125
2656 :  705.0288696289062
2657 :  704.9932861328125
2658 :  704.9569702148438
2659 :  704.91943359375
2660 :  704.8812866210938
2661 :  704.8429565429688
2662 :  704.805419921875
2663 :  704.7681884765625
2664 :  704.7315673828125
2665 :  704.6947631835938
2666 :  704.

3269 :  685.777099609375
3270 :  685.7514038085938
3271 :  685.7254028320312
3272 :  685.6993408203125
3273 :  685.6732177734375
3274 :  685.6470947265625
3275 :  685.6207885742188
3276 :  685.5938110351562
3277 :  685.5668334960938
3278 :  685.5396118164062
3279 :  685.5122680664062
3280 :  685.4849853515625
3281 :  685.4573974609375
3282 :  685.430419921875
3283 :  685.4034423828125
3284 :  685.3764038085938
3285 :  685.3501586914062
3286 :  685.3256225585938
3287 :  685.3016357421875
3288 :  685.2774658203125
3289 :  685.2540283203125
3290 :  685.2310180664062
3291 :  685.2089233398438
3292 :  685.1878662109375
3293 :  685.168701171875
3294 :  685.1512451171875
3295 :  685.1353759765625
3296 :  685.1207885742188
3297 :  685.1041870117188
3298 :  685.0833740234375
3299 :  685.0546264648438
3300 :  685.018310546875
3301 :  684.9776611328125
3302 :  684.9404907226562
3303 :  684.9111328125
3304 :  684.8901977539062
3305 :  684.8734741210938
3306 :  684.8560791015625
3307 :  684.8341064

3913 :  674.0436401367188
3914 :  674.0106201171875
3915 :  674.0381469726562
3916 :  674.0726928710938
3917 :  674.064453125
3918 :  674.0149536132812
3919 :  673.9658813476562
3920 :  673.9515380859375
3921 :  673.9658203125
3922 :  673.9771728515625
3923 :  673.9623413085938
3924 :  673.9287109375
3925 :  673.900146484375
3926 :  673.8914794921875
3927 :  673.8958740234375
3928 :  673.895751953125
3929 :  673.8812255859375
3930 :  673.85791015625
3931 :  673.8387451171875
3932 :  673.8303833007812
3933 :  673.828369140625
3934 :  673.8234252929688
3935 :  673.8109130859375
3936 :  673.7938232421875
3937 :  673.779052734375
3938 :  673.7692260742188
3939 :  673.7635498046875
3940 :  673.7568359375
3941 :  673.7462158203125
3942 :  673.7330322265625
3943 :  673.7198486328125
3944 :  673.7095336914062
3945 :  673.701416015625
3946 :  673.69384765625
3947 :  673.684326171875
3948 :  673.6732177734375
3949 :  673.6614990234375
3950 :  673.6506958007812
3951 :  673.6414184570312
3952 :  6

4553 :  668.9933471679688
4554 :  669.0062255859375
4555 :  669.0107421875
4556 :  668.9969482421875
4557 :  668.9736328125
4558 :  668.9566040039062
4559 :  668.9528198242188
4560 :  668.9566650390625
4561 :  668.9570922851562
4562 :  668.9483032226562
4563 :  668.9335327148438
4564 :  668.92041015625
4565 :  668.9140625
4566 :  668.9126586914062
4567 :  668.910888671875
4568 :  668.9047241210938
4569 :  668.8948974609375
4570 :  668.884521484375
4571 :  668.8768310546875
4572 :  668.8722534179688
4573 :  668.8688354492188
4574 :  668.8637084960938
4575 :  668.8565673828125
4576 :  668.8480224609375
4577 :  668.8405151367188
4578 :  668.83447265625
4579 :  668.8296508789062
4580 :  668.82470703125
4581 :  668.818603515625
4582 :  668.8117065429688
4583 :  668.804443359375
4584 :  668.7978515625
4585 :  668.7920532226562
4586 :  668.7865600585938
4587 :  668.7808837890625
4588 :  668.77490234375
4589 :  668.7682495117188
4590 :  668.7616577148438
4591 :  668.755126953125
4592 :  668.74

In [36]:
# %%time
# prevLoss = 10000.0
# count = 0
# for epoch in range(2000):
#     loss = train(optimizer001, dataset)
#     if(prevLoss - loss<0.1):
#         count+=1
#         if(count>10):
#             print("Converged", prevLoss, ",",loss)
#             if(loss<300):
#                 break
#             else:
#                 count=0
#     else:
#         prevLoss = loss
#     print(loss,epoch)
#     if(loss<10):
#         print("Loss Lim")
#         break

# optimizer = optimizer01
# lr = 0.01
# epoch = 0
# while((dcount<=5)|(prevLoss-loss<0.01)):
#     if(loss<2500):
#         optimizer = optimizer001
#         lr = 0.001
#     if(loss>2500):
#         optimizer = optimizer01
#         lr = 0.01
#     loss = train(optimizer, dtset)
#     if(prevLoss - loss<0.1):
#         count+=1
#         if(count>10):
#             print("Converged", prevLoss, ",",loss)
#             if(loss<300):
#                 dcount+=1
#                 dtset = createDataset(X,edges)
#                 optimizer = optimizer001
#             else:
#                 count=0
#     prevLoss = loss
#     print(loss,epoch,",",dcount,", Opt: ",lr)
#     if(loss<10):
#         print("Loss Lim")
#         break
#     epoch+=1

In [37]:
sumNN, sumMean = 0, 0
#graph = Data(x = torch.from_numpy(X), edge_index = torch.from_numpy(edges[:2,:]).type(torch.LongTensor), norm = findNorm(torch.from_numpy(edges.T),len(X)))
outMean = Meaner(graph)

(2, 24608) (24608,) (2350,)


In [38]:
for i in range(50):
    out = net(dataset[i])
    out = torch.reshape(out,(-1,))
    lossOut = F.mse_loss(out, X1)**0.5
    lossOutVal = F.mse_loss(out[dataset[i].train_set.type(torch.LongTensor)], X1[dataset[i].train_set.type(torch.LongTensor)])**0.5
    meanLossOut = F.mse_loss(outMean, X2)**0.5
    meanLossOutVal = F.mse_loss(outMean[dataset[i].train_set.type(torch.LongTensor)], X2[dataset[i].train_set.type(torch.LongTensor)])**0.5
    sumNN+=lossOutVal.item()
    sumMean+=meanLossOutVal.item()
    print("Iter ", i)
    print("On Whole Set: ", lossOut.item(),"(NN), ", meanLossOut.item(),"(Just Mean)")
    print("On Train Set: ", lossOutVal.item(),"(NN), ", meanLossOutVal.item(),"(Just Mean)")
    print(" ")
print("Total Loss by NN: ", sumNN/50)
print("Total Loss by MeanEstimate: ", sumMean/50)

Iter  0
On Whole Set:  4.011395407585009 (NN),  5.192622684353297 (Just Mean)
On Train Set:  12.568275259676565 (NN),  5.349666823523876 (Just Mean)
 
Iter  1
On Whole Set:  2.7903743297126176 (NN),  5.192622684353297 (Just Mean)
On Train Set:  8.646133162149452 (NN),  4.429265614742525 (Just Mean)
 
Iter  2
On Whole Set:  3.0446614759873443 (NN),  5.192622684353297 (Just Mean)
On Train Set:  9.460485156543122 (NN),  4.739082835300558 (Just Mean)
 
Iter  3
On Whole Set:  3.6513511781588224 (NN),  5.192622684353297 (Just Mean)
On Train Set:  11.405430124010847 (NN),  4.923702611264904 (Just Mean)
 
Iter  4
On Whole Set:  4.024838207649399 (NN),  5.192622684353297 (Just Mean)
On Train Set:  12.61031245982811 (NN),  5.156922551884623 (Just Mean)
 
Iter  5
On Whole Set:  3.1597340748836324 (NN),  5.192622684353297 (Just Mean)
On Train Set:  9.832538951949354 (NN),  4.826816009432484 (Just Mean)
 
Iter  6
On Whole Set:  4.080853296307562 (NN),  5.192622684353297 (Just Mean)
On Train Set:  1

In [39]:
k = 46
dataset[k].x.T[0][:50], net(dataset[k])[:50], X[:50], outMean[:50]

(tensor([ 83.0000,  86.0000,  78.7100,  83.7900,  82.2000,  86.6000,  84.5400,
          88.9200,  85.5000,  89.5000,  88.0000,  83.8600,  82.1300,  82.6400,
          78.4400,  81.5000,  79.6700,   0.0000,  80.6700,  79.7100,  77.0000,
           0.0000,  81.9200,  74.6300,  84.1000,  78.1200,  76.9100,  82.6400,
          88.1900, 102.0500, 122.8600,  77.8400,  83.7300,  79.0800,  76.7100,
           0.0000,  76.7100,  82.7500,  78.0000,  82.7700,  83.0000,  75.1700,
          76.0800,  82.8000,  79.8000,  84.5000,   0.0000,  85.2200,   0.0000,
          87.5000], device='cuda:0'),
 tensor([ 82.8165,  85.8913,  78.6648,  83.5741,  81.9161,  86.6780,  84.6235,
          89.0201,  85.8648,  89.5278,  87.7171,  84.1466,  82.1820,  82.3454,
          78.6047,  81.2285,  79.8216,  81.1232,  80.5467,  79.5658,  76.8854,
          79.9562,  82.0816,  74.4639,  83.9128,  78.1629,  76.8272,  81.7581,
          87.5002, 101.9408, 123.6032,  77.6960,  83.4477,  79.0605,  76.3643,
          79.6

In [40]:
# %%time
# from haversine import haversine, Unit

# #----------------------------------------------2-3Km in space, 1-2.5hrs in time----------------------------------------
# spaceD = data[:,1:3]
# timeD = data[:,0]
# dist = 1.0 #in Km, Time
# edges = []
# for i in range(len(data)):
#     lim = 50
#     count = 0
#     se = []
#     while(count<4):
#         for j in range(len(data)):
#             if(j in se):
#                 continue
#             di = haversine(spaceD[i],spaceD[j])
#             ti = int(abs(timeD[i]-timeD[j])/14.9)
#             w = (1+di)*(1+di)*(1+di)*(1+ti/2)
#             if ((ti<=0) & (w<=lim)):
#                 edges.append([j,i,1.0/w])
#                 count+=1
#                 se.append(j)
#         lim+=50
# edges = np.array(edges).T

In [41]:
def prepareTestData(trainData, testData, edges):
    finalData = np.concatenate((trainData,testData), axis = 0)
    spaceD = finalData[:,1:3]
    timeD = finalData[:,0]
    finalEdges = edges.T
    trainLen = len(trainData)
    newEdges = []
    for i in range(len(testData)):
        lim = 50
        count = 0
        se = []
        while(count<4):
            for j in range(len(trainData)):
                if(j in se):
                    continue
                di = haversine(spaceD[i+trainLen],spaceD[j])
                ti = int(abs(timeD[i+trainLen]-timeD[j])/14.9)
                w = (1+di)*(1+di)*(1+di)*(1+ti/2)
                if ((ti<=0) & (w<=lim)):
                    newEdges.append([j,i+trainLen,1.0/w])
                    count+=1
                    se.append(j)
            lim+=50
    newEdges = np.array(newEdges)
    finalEdges = np.concatenate((finalEdges, newEdges), axis = 0)
    return finalData[:,3],finalEdges.T

In [42]:
Xf, ef = prepareTestData(data, testData, edges)
Xf = np.reshape(Xf,(Xf.shape[0],1))
Xf = np.concatenate((Xf,np.ones((Xf.shape[0],1))), axis = 1)
Xf[-(len(testData)):,:] = 0
test_set = range(X.shape[0],Xf.shape[0])

ei = torch.tensor(ef[:2,:], dtype= torch.long)
nm = findNorm(ef.T, Xf.shape[0])
model = Data(x = torch.from_numpy(Xf), edge_index = ei,
             test_set = test_set, norm = nm)
model.to(device)
net.eval()
out = net(model)
outMean = Meaner(model)
tdt = torch.from_numpy(testData[:,2])
print("Loss by GNN model: ", F.mse_loss(out[test_set], tdt.to(device)))
print("Loss by mean model: ", F.mse_loss(outMean[test_set], tdt))

(2, 25842) (25842,) (2473, 2)
Loss by GNN model:  tensor(796.3347, device='cuda:0', dtype=torch.float64,
       grad_fn=<MseLossBackward>)
Loss by mean model:  tensor(855.8314, dtype=torch.float64)


In [44]:
print((sum((outMean[test_set] - testData[:,3])**2)/len(testData)).item()**0.5)
print((sum((out[test_set].to('cpu').detach().numpy() - testData[:,3])**2)/len(testData))**0.5)

7.867835673379697
7.566443052343096


In [34]:
data[ef[:,ef[1] == 2472][0].astype(int), :]

array([[390.   ,  28.55 ,  77.265,  87.58 ],
       [390.   ,  28.55 ,  77.275,  89.58 ],
       [390.   ,  28.555,  77.265,  86.14 ],
       [390.   ,  28.555,  77.27 ,  95.   ],
       [390.   ,  28.555,  77.275,  86.   ],
       [390.   ,  28.56 ,  77.255,  83.   ],
       [390.   ,  28.56 ,  77.26 ,  80.38 ],
       [390.   ,  28.56 ,  77.265,  85.5  ],
       [390.   ,  28.565,  77.25 ,  85.29 ],
       [390.   ,  28.565,  77.255,  77.33 ]])

In [35]:
testData

array([[ 975.   ,   28.625,   77.24 ,   31.27 ],
       [ 750.   ,   28.605,   77.24 ,   97.   ],
       [ 495.   ,   28.565,   77.235,  108.91 ],
       [ 825.   ,   28.6  ,   77.165,   44.5  ],
       [ 300.   ,   28.56 ,   77.255,   85.89 ],
       [ 510.   ,   28.595,   77.165,   85.09 ],
       [1320.   ,   28.6  ,   77.24 ,   93.75 ],
       [ 825.   ,   28.545,   77.275,   64.5  ],
       [ 510.   ,   28.605,   77.155,   87.27 ],
       [ 465.   ,   28.655,   77.29 ,  116.22 ],
       [1170.   ,   28.56 ,   77.27 ,   67.33 ],
       [ 480.   ,   28.635,   77.285,  110.66 ],
       [ 660.   ,   28.525,   77.295,   92.33 ],
       [ 405.   ,   28.555,   77.27 ,  114.5  ],
       [ 795.   ,   28.565,   77.25 ,   64.15 ],
       [1140.   ,   28.575,   77.21 ,   38.71 ],
       [1260.   ,   28.505,   77.3  ,   90.17 ],
       [1200.   ,   28.63 ,   77.24 ,  107.96 ],
       [ 615.   ,   28.555,   77.27 ,   80.73 ],
       [ 945.   ,   28.555,   77.265,   30.83 ],
       [ 885.   ,   